In [9]:
require(tidyverse)
# install.packages('hoopR')
require(hoopR)
require(lubridate)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Loading required package: lubridate


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [10]:
years.vec = (2003:2022) %>% setdiff(., 2020)
start.dates.vec = as.Date(c('2003-03-18','2004-03-16', '2005-03-15', '2006-03-14', '2007-03-13', '2008-03-18', '2009-03-17', '2010-03-16', '2011-03-15', '2012-03-13', '2013-03-19', '2014-03-18', '2015-03-17', '2016-03-15', '2017-03-14', '2018-03-13', '2019-03-19', '2021-03-18', '2022-03-13'))

rip.df = data.frame(years.vec, start.dates.vec)

for (what.year in years.vec){

date.str = rip.df %>% filter(years.vec == what.year) %>% pull(start.dates.vec) %>% as.Date()

####

box.23 = hoopR::load_mbb_player_box(what.year) %>% mutate(pts = as.numeric(pts))

sched.23 = hoopR::load_mbb_schedule(what.year) %>% select(season, date, home_short_display_name, away_short_display_name, home_score, away_score, home_winner, home_logo, away_logo) %>% drop_na() %>% mutate(date = as.Date(date))

opponent.df = rbind(sched.23 %>% select(season, date, 'team'=home_short_display_name, "opponent" = away_short_display_name), sched.23 %>% select(season, date, 'team'=away_short_display_name, "opponent" = home_short_display_name))

box.23 = box.23 %>% 
  mutate(min = as.numeric(min)) %>% 
  mutate(oreb = as.numeric(oreb)) %>% 
  mutate(dreb = as.numeric(dreb)) %>% 
  mutate(ast = as.numeric(ast)) %>% 
  mutate(stl = as.numeric(stl)) %>% 
  mutate(blk = as.numeric(blk)) %>% 
  mutate(to = as.numeric(to)) %>% 
  mutate(pf = as.numeric(pf)) %>% 
  separate(ft, into=c('ft_made','ft_attempted'), sep='-') %>%
  mutate(ft_attempted = as.numeric(ft_attempted), ft_made = as.numeric(ft_made)) %>%
  separate(fg3, into=c('tpt_made','tpt_attempted'), sep='-') %>%
  mutate(tpt_made = as.numeric(tpt_made), tpt_attempted = as.numeric(tpt_attempted)) %>% 
  rename('team' = team_short_display_name) %>% 
  rename('date' = game_date) %>% 
  separate(fg, into=c('fg_made','fg_attempted'), sep='-') %>% 
  mutate(fg_attempted = as.numeric(fg_attempted), fg_made = as.numeric(fg_made)) 

box.23 = box.23 %>% inner_join(., opponent.df, by=c('date','team'))

# only since february

box.23 = box.23 %>% mutate(month = month(date)) %>% filter(month == 2 | month == 3)

# shooting pct, three point percent, minutes HHI, points HHI, offensive rebound rate, defensive rebound rate, assists per points, turnovers per point, total team fouls

# shooting percent

shooting_pct = box.23 %>% group_by(season.x, team) %>% summarize(fg_pct = sum(fg_made)/sum(fg_attempted), three_pct = sum(tpt_made)/sum(tpt_attempted))
shooting_pct_against = box.23 %>% group_by(season.x, opponent) %>% summarize(d_fg_pct = sum(fg_made)/sum(fg_attempted), d_three_pct = sum(tpt_made)/sum(tpt_attempted)) %>% rename("team"='opponent')

ft_pct = box.23 %>% group_by(season.x, team) %>% summarize(ft_pct = sum(ft_made)/sum(ft_attempted))

# minutes HHI

min.hhi = inner_join(box.23 %>% group_by(season.x, team, athlete_display_name) %>% summarize(total_minutes = sum(min)), box.23 %>% group_by(season.x, team) %>% summarize(team_minutes = sum(min))) %>% mutate(min_pct = total_minutes / team_minutes) %>% group_by(season.x, team) %>% summarize(total.hhi = sum(min_pct^2))

# points hhi

pts.hhi = inner_join(box.23 %>% group_by(season.x, team, athlete_display_name) %>% summarize(total_points = sum(pts)), box.23 %>% group_by(season.x, team) %>% summarize(team_points = sum(pts))) %>% mutate(pts_pct = total_points / team_points) %>% group_by(season.x, team) %>% summarize(total.points.hhi = sum(pts_pct^2))

# offensive rebound rate 

oreb.rate = box.23 %>% group_by(season.x, team) %>% summarize(oreb_rate = sum(oreb)/(sum(ft_attempted)+sum(tpt_attempted)))
dreb.rate = box.23 %>% group_by(season.x, opponent) %>% summarize(dreb_rate = sum(dreb)/(sum(ft_attempted)+sum(tpt_attempted)))%>% rename("team"='opponent')

# assists per point

assists.per.pts = box.23 %>% group_by(season.x, team) %>% summarize(assists_per_point = sum(ast)/sum(pts))

# turnovers per point

to.committed.gm = box.23 %>% group_by(season.x, team) %>% summarize(to.commit = sum(to)/n_distinct(date))
to.forced.gm = box.23 %>% group_by(season.x, opponent) %>% summarize(to.commit = sum(to)/n_distinct(date))%>% rename("team"='opponent')

# total team fouls

total.fouls.gm = box.23 %>% group_by(season.x, team) %>% summarize(total_fouls = sum(pf)/n_distinct(date))
total.fouls.against.gm = box.23 %>% group_by(season.x, opponent) %>% summarise(fouls_against=sum(pf)/n_distinct(date))%>% rename("team"='opponent')

# combining all of them

df_list = list(shooting_pct, shooting_pct_against, ft_pct, min.hhi, pts.hhi, oreb.rate, dreb.rate, to.committed.gm, to.forced.gm, assists.per.pts, total.fouls.gm, total.fouls.against.gm)

# took out: 


big.df = df_list %>% reduce(inner_join, by=c('season.x','team'))

### relevant schedule

trunc.sched = load_mbb_schedule(what.year) %>% mutate(date = as.Date(date)) %>% filter(date>=date.str) %>% select(date, home_short_display_name, away_short_display_name, home_score, away_score) %>% mutate(home_win = ifelse(home_score>away_score, 1, 0)) %>% mutate(season.x = year(date))

temp.trunc = inner_join(trunc.sched, big.df, by=c('season.x','home_short_display_name'='team'))

final.df = inner_join(temp.trunc, big.df, by=c('season.x','away_short_display_name'='team')) %>% select(-season.x)

if(what.year == min(years.vec)){
  output.df = final.df
} else {output.df = rbind(output.df, final.df)}

}


`summarise()` has grouped output by 'season.x'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'season.x'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'season.x'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'season.x', 'team'. You can override using
the `.groups` argument.
`summarise()` has grouped output by 'season.x'. You can override using the
`.groups` argument.
Joining with `by = join_by(season.x, team)`
`summarise()` has grouped output by 'season.x'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'season.x', 'team'. You can override using
the `.groups` argument.
`summarise()` has grouped output by 'season.x'. You can override using the
`.groups` argument.
Joining with `by = join_by(season.x, team)`
`summarise()` has grouped output by 'season.x'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'seas

In [13]:
require(stringr)
project_name = 'ML-MADNESS'
repo_path = file.path(substr(getwd(),0,gregexpr(pattern = project_name ,
                                                getwd())[[1]][1]-2),
                      project_name)
setwd(repo_path)

In [ ]:
source(file.path(repo_path,'objects','helper_functions.R'))
p = scan(file.path(repo_path,'features/requirements.txt'),what="",sep="\n")
load_all_packages(p)
p = scan(file.path(repo_path,'sagemaker/requirements.txt'),what="",sep="\n")
load_all_packages(p)
slice = dplyr::slice
rename = dplyr::rename


Attaching package: ‘progressr’


The following object is masked from ‘package:httr’:

    progress



Attaching package: ‘memoise’


The following object is masked from ‘package:httr’:

    timeout


also installing the dependencies ‘paws.common’, ‘lgr’, ‘paws.storage’




In [ ]:
# Sagemaker Settings
sagemaker <- import('sagemaker')
session <- sagemaker$Session()
bucket <- session$default_bucket()
role_arn <- sagemaker$get_execution_role()

In [ ]:
predictors = c("PTS_SCORED_ELO",
              "PTS_ALLOWED_ELO",
              "FG3M_ELO",
              "FG3M_ALLOWED_ELO",
              "OFFENSIVE_REBOUNDS_ELO",
              "DEFENSIVE_REBOUNDS_ELO",
              "TURNOVERS_ELO",
              "TURNOVERS_FORCED_ELO",
               "PTS_SCORED_DEF_ELO",
              "PTS_ALLOWED_DEF_ELO",
              "FG3M_DEF_ELO",
              "FG3M_ALLOWED_DEF_ELO",
              "OFFENSIVE_REBOUNDS_DEF_ELO",
              "DEFENSIVE_REBOUNDS_DEF_ELO",
              "TURNOVERS_DEF_ELO",
              "TURNOVERS_FORCED_DEF_ELO"
              )
non_predictors = c('game_date','game_id','team_id','opponent_id')

pre_modeling = df %>% 
            filter(APPEARANCE!=1,
            OPP_APPEARANCE!=1) %>%
            #select(PTS.x,PTS.y,game_date,game_id,team_id,opponent_id,predictors)%>%
            mutate(MARGIN = PTS.x - PTS.y,
                   WIN = ifelse(MARGIN >0,1,0))
modeling = pre_modeling
head(modeling)